# 文本生成

## 样例说明
本文展示了如何使用appbuilder内置组件以及如何快速构建全新的文本生成组件。以下是一个在营销场景下生成各种文本内容的样例。

## 概览
具体地，本样例按照以下流程进行文本生成：
1. 使用**空模板**构建一个全新的文本生成组件：商品信息生成组件。
2. 输入一个商品，使用新构建的商品信息生成组件生成该商品的商品信息。
3. 基于生成的商品信息，使用内置的**问答对挖掘**组件生成问答对。生成的问答对可用于客服等。
4. 基于生成的商品信息，使用内置的**风格写作**组件为该商品生成一份【小红书】文案。
5. 使用内置的**标签抽取**组件对生成的文案进行标签抽取。抽取出的标签可用于检索文案。

以下我们会演示如何实现我们自己的定制化能力。

## 流程演示
### Step 0: 安装Python SDK
我们的appbuilder支持使用pip安装：

In [ ]:
pip install appbuilder

代码中需要配置用户的`APPBUILDER_TOKEN`和`GATEWAY_URL`。

In [11]:
import os

# 设置环境变量
os.environ['APPBUILDER_TOKEN'] = '...'
os.environ['GATEWAY_URL'] = '...'

安装成功后，我们就可以搭建我们自己的能力了。

### Step 1: 商品信息生成
基于appbuilder提供的**空模板**，用户可以自定义文本生成组件以解决多样化的需求。这里我们构建一个**商品信息生成**组件以生成商品信息：

In [ ]:
from appbuilder import Message, Playground

# 输入到大模型中的prompt的模板
prompt_template = \
'''输入商品名，我需要你为我生成该商品的商品信息。

要求：
- 直接输出商品信息，不需要引导语。

商品名：{product_name}
商品信息：
'''
# 创建商品信息生成组件
product_information_generation = Playground(prompt_template=prompt_template, model_name='ernie-bot-4')

# 获取商品信息
# 填充prompt_template参数的参数映射表，需要与prompt_template对应
prompt_template_kwargs = {
    # 'product_name': 'IPhone 14 Pro'
    'product_name': '特斯拉Model Y'
}
response = product_information_generation(Message(prompt_template_kwargs), stream=False, temperature=0.5)
product_information = response.content
print(f'商品信息：\n{product_information}')

输出结果：
```
商品信息：
车型：特斯拉Model Y
价格：约29-40万人民币
尺寸：中型SUV
续航里程：NEDC工况下可达594公里
加速性能：0-100km/h加速时间仅为3.7秒
充电时间：超级充电站30分钟可充80%电量
驱动方式：后置后驱或双电机四驱
座位数：5座或7座可选
颜色：多种颜色可选
特点：
- 时尚动感的外观设计
- 宽敞舒适的内部空间
- 强大的性能和续航能力
- 丰富的智能科技配置
- 特斯拉独有的自动驾驶技术

特斯拉Model Y是一款中型SUV，拥有时尚动感的外观设计和宽敞舒适的内部空间。它搭载了强大的电动机和电池组，能够提供出色的性能和续航能力。此外，Model Y还配备了丰富的智能科技配置和特斯拉独有的自动驾驶技术，为用户带来更加便捷、安全和智能的驾驶体验。
```

### Step 2: 问答对生成
基于Step 1生成的商品信息我们生成一些问答对，这里我们使用内置的**问答对生成**组件。

In [ ]:
from appbuilder import QAPairMining

# 初始化问答对生成组件
qa_pair_mining = QAPairMining(model_name='ernie-bot-apaas')

# 获取问答对
response = qa_pair_mining(Message(product_information), stream=False, temperature=1e-10)
qa_pairs = response.content
print(f'问答对：\n{qa_pairs}')

输出结果：
```
问答对：
1. 特斯拉Model Y的车型是什么？
2. 特斯拉Model Y的价格大约是多少？
3. 特斯拉Model Y的尺寸是多少？
4. 特斯拉Model Y的续航里程可以达到多少？
5. 特斯拉Model Y的0-100km/h加速时间是多少？
6. 特斯拉Model Y的充电时间是多少？
7. 特斯拉Model Y的驱动方式是什么？
8. 特斯拉Model Y的座位数是多少？
9. 特斯拉Model Y有哪些颜色可选？
10. 特斯拉Model Y的特点有哪些？
```

### Step 3: 营销文案生成
基于Step 1生成的商品信息生成【小红书】文案，这里我们使用内置的**风格写作**组件。

In [ ]:
from appbuilder import StyleWriting

# 构造query
query = \
f'''请你基于以下商品信息生成文案：
{product_information}
'''

# 初始化风格写作组件
style_writing = StyleWriting(model_name='ernie-bot-4')

# 获取小红书文案
response = style_writing(Message(query), style_query='小红书', length=300)
copywriting = response.content
print(f'文案：\n{copywriting}')

输出结果：
```
文案：
【特斯拉Model Y，中型SUV的新标杆！】

🚀 特斯拉Model Y，中型SUV界的翘楚！时尚动感的外观、宽敞舒适的内部空间，让你一见倾心！💓

🚀 强大的性能和续航能力，让你畅享驾驶乐趣！0-100km/h加速仅需3.7秒，NEDC工况下续航里程可达594公里！🔋

🚀 充电也无忧！特斯拉超级充电站30分钟可充80%电量，让你的时间更加宝贵！🔌

🚀 后置后驱或双电机四驱，5座或7座可选，满足你的各种需求！多种颜色可选，让你的Model Y更加个性十足！🌈

🚀 丰富的智能科技配置和特斯拉独有的自动驾驶技术，让驾驶更加便捷、安全和智能！💡

#特斯拉ModelY #中型SUV #强大性能 #续航能力 #智能科技 #自动驾驶技术
```

### Step 4: 标签抽取
基于Step 3生成的文案进行标签抽取，这里我们使用内置的**标签抽取**组件。

In [ ]:
from appbuilder import TagExtraction

# 初始化标签抽取组件
tag_extraction = TagExtraction(model_name='ernie-bot-4')

# 获取标签
response = tag_extraction(Message(copywriting), stream=False, temperature=1e-10)
tags = response.content
print(f'标签：\n{tags}')

输出结果：
```
标签：
1.特斯拉Model Y
2.中型SUV
3.强大性能
4.续航能力
5.充电无忧
6.多种选择
7.个性十足
8.智能科技
9.自动驾驶
10.时尚动感
```